In [1]:
!pip install transformers[torch]

     |████████████████████████████████| 675kB 2.7MB/s 
     |████████████████████████████████| 1.1MB 13.1MB/s 
     |████████████████████████████████| 890kB 12.3MB/s 
     |████████████████████████████████| 3.8MB 25.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=0c1ba70f1404b9282d0c2cfaf30d5c4ceed219d4758b34d9e79e3f4f5da96eea
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


# Summarization con Google T5

In [2]:
import torch
import transformers

Hacemos uso de la librería de huggingface de transformers. Para más información visitart https://huggingface.co/transformers/

# Sumarization

In [3]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from transformers import AutoModelWithLMHead, AutoTokenizer
from transformers import TFAutoModelWithLMHead, AutoTokenizer



In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [5]:
path = '/content/drive/Shared drives/NLP/Project NLP/Final/'

Aquí filtramos algunos documentos que no cumplen con ciertas características para generar sus correspondientes resúmenes

In [6]:
'''
Only summarize when all the following conditions are met:
    * len(text) > 100.
    * there is an abstract available and abstract > 100
'''
df = pd.read_csv(path + 'preprocessed.csv')
df.dropna(subset=['text', 'abstract'], inplace=True)
df.drop(df[(df['abstract'].str.len() < 100) | (df['text'].str.len() < 100)].index, inplace=True)

Estos son los modelos preentrenados de huggingface que vamos a utilizar.

In [7]:
model = TFAutoModelWithLMHead.from_pretrained("t5-base")
tokenizer = AutoTokenizer.from_pretrained("t5-base")

In [ ]:
sums = []

for i in tqdm(range(len(df))):
    text = df["text"].iloc[i]
    abstract = df["abstract"].iloc[i]
    if (type(text) is not str) or (len(text) < 100):
        sums.append("")
        continue
    if (type(abstract) is not str) or (len(abstract) < 100):
        sums.append("")
        continue
    # T5 uses a max_length of 512 so we cut the article to 512 tokens.
    inputs = tokenizer.encode("summarize: " + text, return_tensors="tf", max_length=512)
    outputs = model.generate(inputs, max_length=300, min_length=100, length_penalty=2.0, num_beams=4, early_stopping=True)
    #print(outputs)
    summary = tokenizer.decode(outputs[0].numpy())
    sums.append(summary)
    
    with open( path + "/summaries/" + df["pmcid"].iloc[i] + ".txt", "a") as f: 
      f.write(summary)
      f.close()

 56%|█████▌    | 2168/3901 [9:54:45<8:38:33, 17.95s/it]